# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import sys

#### Creating list of filepaths to process original event csv data files

In [2]:
# check platform to determine if running jupyter notebook locally on a windows machine or a non-windows machine
# change directory path accordingly
platform = sys.platform
if platform == "win32":
    chdir_path = 'D:/Documents/GitHub/Udacity/Data-Engineering-Nanodegree/Data-Modeling-with-Cassandra'
    os.chdir(chdir_path)
    

In [3]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-24-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. if the number of rows in the csv file equals 6821 proceed to the next cell... 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Create a Cluster

In [6]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Use Panda dataframes to add columns to the query output

In [9]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra we model the database tables on the queries we want to run.

## Sparkify would like the following questions/queries answered:

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### QUERY 1

In [10]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "DROP TABLE IF EXISTS artist_song_session"
try:
    session.execute(query)
except Exception as e:
    print(e)  

# rule of thumb... primary key with or without clustering columns must be unique
# primary key will usually contain the columns that are in the where clause of the query
# in this query, the primary key will be session_id and item_in_session which is necessary in order to satisfy uniqueness
query = "CREATE TABLE IF NOT EXISTS artist_song_session "
query = query + """
(session_id int
,item_in_session int
,artist_name text
,song_title text
,song_length float
,user_id int
,user_first_name text
,user_last_name text
,PRIMARY KEY (session_id, item_in_session))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)                 

In [11]:
## insert into artist_song_session table

# line[0] = artist
# line[1] = firstName
# line[2] = gender
# line[3] = itemInSession
# line[4] = lastName
# line[5] = length
# line[6] = level
# line[7] = location
# line[8] = sessionId
# line[9] = song
# line[10] = userId

file = 'event_datafile_new.csv'

query = """
INSERT INTO artist_song_session 
(session_id
,item_in_session
,artist_name
,song_title
,song_length
)
"""
query = query + " VALUES (%s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line[5])
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## Verify
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = """
select artist_name
,song_title
,song_length 
from artist_song_session 
where session_id = 338 
and item_in_session = 4
"""
try:
    rslt = session.execute(query, timeout=None)
    df = rslt._current_rows
    print(df)
except Exception as e:
    print(e)                  

  artist_name                       song_title  song_length
0   Faithless  Music Matters (Mark Knight Dub)   495.307312


#### QUERY 2

In [15]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "DROP TABLE IF EXISTS artist_song_user"
try:
    session.execute(query)
except Exception as e:
    print(e)  
    
# rule of thumb... primary key with or without clustering columns must be unique
# primary key will usually contain the columns that are in the where clause of the query
# in this query, the composite primary key will be user_id and session_id
# item_in_session will be the clustering key - The Clustering Key is responsible for data sorting within the partition.
query = "CREATE TABLE IF NOT EXISTS artist_song_user "
query = query + """
(user_id int
,session_id int
,item_in_session int
,artist_name text
,song_title text
,song_length float
,user_first_name text
,user_last_name text
,PRIMARY KEY ((user_id, session_id), item_in_session))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)                 
                    

In [16]:
## insert into artist_song_user table

# line[0] = artist
# line[1] = firstName
# line[2] = gender
# line[3] = itemInSession
# line[4] = lastName
# line[5] = length
# line[6] = level
# line[7] = location
# line[8] = sessionId
# line[9] = song
# line[10] = userId

file = 'event_datafile_new.csv'

query = """
INSERT INTO artist_song_user 
(session_id
,item_in_session
,artist_name
,song_title
,song_length
,user_id
,user_first_name
,user_last_name
)
"""
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]), int(line[10]), line[1], line[4]))

In [17]:
## Verify
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = """
select artist_name
,song_title
,user_first_name
,user_last_name 
from artist_song_user 
where user_id = 10 
and session_id = 182
"""
try:
    rslt = session.execute(query, timeout=None)
    df = rslt._current_rows
    print(df)
except Exception as e:
    print(e)                  

         artist_name                                         song_title  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  user_first_name user_last_name  
0          Sylvie           Cruz  
1          Sylvie           Cruz  
2          Sylvie           Cruz  
3          Sylvie           Cruz  


#### QUERY 3

In [18]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "DROP TABLE IF EXISTS user_song"
try:
    session.execute(query)
except Exception as e:
    print(e)  
    
# rule of thumb... primary key with or without clustering columns must be unique
# primary key will usually contain the columns that are in the where clause of the query
# in this query, the primary key will be song_title, user_id which is necessary in order to satisfy uniqueness
query = "CREATE TABLE IF NOT EXISTS user_song "
query = query + """
(song_title text
,user_id int
,user_first_name text
,user_last_name text
,PRIMARY KEY (song_title, user_id))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)                 


In [19]:
## insert into user_song table

# line[0] = artist
# line[1] = firstName
# line[2] = gender
# line[3] = itemInSession
# line[4] = lastName
# line[5] = length
# line[6] = level
# line[7] = location
# line[8] = sessionId
# line[9] = song
# line[10] = userId

file = 'event_datafile_new.csv'

query = """
INSERT INTO user_song 
(song_title
,user_id
,user_first_name
,user_last_name)
"""
query = query + " VALUES (%s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [20]:
## Verify
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "select user_first_name, user_last_name from user_song where song_title = 'All Hands Against His Own'"
try:
    rslt = session.execute(query, timeout=None)
    df = rslt._current_rows
    print(df)
except Exception as e:
    print(e)                  

  user_first_name user_last_name
0      Jacqueline          Lynch
1           Tegan         Levine
2            Sara        Johnson


### Drop the tables before closing out the sessions

In [21]:
## Drop the table before closing out the sessions

query = "DROP TABLE IF EXISTS artist_song_session"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS artist_song_user"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()